In [1]:
import sqlite3

# Conexión a la base de datos SQLite
conexion = sqlite3.connect("tpv_peluqueria.db")
cursor = conexion.cursor()

# Crear tablas
cursor.execute("""
CREATE TABLE IF NOT EXISTS empleados (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS productos_servicios (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    tipo TEXT CHECK(tipo IN ('producto', 'servicio')) NOT NULL,
    precio REAL NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS clientes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS ventas (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    fecha DATE NOT NULL,
    id_empleado INTEGER NOT NULL,
    id_cliente INTEGER,
    total REAL NOT NULL,
    forma_pago TEXT CHECK(forma_pago IN ('efectivo', 'tarjeta')) NOT NULL,
    FOREIGN KEY (id_empleado) REFERENCES empleados(id),
    FOREIGN KEY (id_cliente) REFERENCES clientes(id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS detalle_ventas (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    id_venta INTEGER NOT NULL,
    id_producto_servicio INTEGER NOT NULL,
    cantidad INTEGER NOT NULL,
    precio_unitario REAL NOT NULL,
    subtotal REAL NOT NULL,
    FOREIGN KEY (id_venta) REFERENCES ventas(id),
    FOREIGN KEY (id_producto_servicio) REFERENCES productos_servicios(id)
)
""")

# Confirmar cambios y cerrar conexión
conexion.commit()
conexion.close()

print("Base de datos creada correctamente.")


Base de datos creada correctamente.


In [2]:
import sqlite3
from datetime import datetime, timedelta
import random

# Conexión a la base de datos SQLite
conexion = sqlite3.connect("tpv_peluqueria.db")
cursor = conexion.cursor()

# Datos iniciales para empleados
empleados = [("Ana",), ("Luis",), ("Sofía",)]
cursor.executemany("INSERT INTO empleados (nombre) VALUES (?)", empleados)

# Datos iniciales para productos y servicios
productos_servicios = [
    ("Corte de pelo", "servicio", 15.0),
    ("Tinte", "servicio", 30.0),
    ("Peinado", "servicio", 20.0),
    ("Mascarilla capilar", "producto", 12.0),
    ("Champú profesional", "producto", 18.0),
    ("Acondicionador reparador", "producto", 10.0)
]
cursor.executemany(
    "INSERT INTO productos_servicios (nombre, tipo, precio) VALUES (?, ?, ?)",
    productos_servicios
)

# Datos iniciales para clientes
clientes = [("María",), ("Carlos",), ("Laura",)]
cursor.executemany("INSERT INTO clientes (nombre) VALUES (?)", clientes)

# Generar datos de ventas anteriores
num_ventas = 10
for _ in range(num_ventas):
    fecha = (datetime.now() - timedelta(days=random.randint(1, 30))).strftime("%Y-%m-%d")
    id_empleado = random.randint(1, len(empleados))
    id_cliente = random.choice([None, random.randint(1, len(clientes))])
    forma_pago = random.choice(["efectivo", "tarjeta"])
    
    # Agregar venta principal
    cursor.execute("""
    INSERT INTO ventas (fecha, id_empleado, id_cliente, total, forma_pago)
    VALUES (?, ?, ?, ?, ?)
    """, (fecha, id_empleado, id_cliente, 0, forma_pago))
    id_venta = cursor.lastrowid

    # Generar detalles de venta
    num_items = random.randint(1, 3)
    total_venta = 0
    for _ in range(num_items):
        id_producto_servicio = random.randint(1, len(productos_servicios))
        cantidad = random.randint(1, 2)
        cursor.execute("SELECT precio FROM productos_servicios WHERE id = ?", (id_producto_servicio,))
        precio_unitario = cursor.fetchone()[0]
        subtotal = cantidad * precio_unitario
        total_venta += subtotal

        cursor.execute("""
        INSERT INTO detalle_ventas (id_venta, id_producto_servicio, cantidad, precio_unitario, subtotal)
        VALUES (?, ?, ?, ?, ?)
        """, (id_venta, id_producto_servicio, cantidad, precio_unitario, subtotal))
    
    # Actualizar el total de la venta
    cursor.execute("UPDATE ventas SET total = ? WHERE id = ?", (total_venta, id_venta))

# Confirmar cambios y cerrar conexión
conexion.commit()
conexion.close()

print("Datos iniciales y ventas generadas correctamente.")


Datos iniciales y ventas generadas correctamente.
